<h1><center> </center></h1>

# Create and plot basic investigation files

In [ ]:
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
import SWOT_Step_2_make_files as step_2
import SOFiA as SOFiA
import cmocean
import pandas as pd

In [ ]:
# import python packages
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
import cartopy as cy
from matplotlib.dates import DateFormatter
import cartopy.crs as ccrs 
import cmocean
import netCDF4 as nc
import glob as glob
import pandas as pd
from scipy.interpolate import griddata
# Binning (Jonathan)
from scipy.stats import binned_statistic_2d
import cartopy.feature as cfeature
import cmcrameri as cmc
# Other
from scipy.fft import fft, fftfreq
from matplotlib.collections import LineCollection
import matplotlib.cm as cm
from matplotlib.patches import Ellipse

from scipy.spatial.distance import cdist
from scipy.linalg import solve
from scipy.signal import butter, filtfilt

import requests
import os
import ftplib
from getpass import getpass
#import cartopy.crs as ccrs
import cartopy.feature as cft
import cartopy.mpl.geoaxes as cmplgeo
import cartopy.mpl.gridliner as cmplgrid
#import matplotlib.pyplot as plt

import SWOT_Step_2_make_files as step_2
import SOFiA as SOFiA

### PART 1 : 

In [ ]:
''' FUNCTIONS PART 1 : '''
'''  1) Count of no-nan in ssh_karin and ssh_karin_2 '''                                             # DIMS: (num_lines,num_pixels)
def count_no_nan(ds, var_type, correction_type, mask_type, smoothing):
    #ds = xr.open_dataset(path_merged+pass_nr+'.nc')
    #masked_d, lat, lon, month_list, year_list, Winter_24, Winter_25, Summer_24 = SOFiA.get_corrected_and_masked_variables2(ds, var_type, correction_type, mask_type, smoothing)
    masked_d, lat, lon, month_list, year_list, Winter_24, Winter_25, Summer_24 = SOFiA.get_corrected_and_masked_variables_C0_C2_all_own_ssha(ds, var_type, correction_type, mask_type, smoothing)
    ''' NUMBER OF GOOD DATA POINTS : '''
    count = np.sum(~np.isnan(masked_d), axis=0)
    return count, lat, lon
'''  2) Standard deviation of ssh_karin_2 without geoid (still tides in it) '''                      # DIMS: (num_lines,num_pixels)
def standard_deviation(ds, var_type, correction_type, mask_type, smoothing):
    #ds = xr.open_dataset(path_merged+pass_nr+'.nc')
    #masked_d, lat, lon, month_list, year_list, Winter_24, Winter_25, Summer_24 = SOFiA.get_corrected_and_masked_variables2(ds, var_type, correction_type, mask_type, smoothing)
    masked_d, lat, lon, month_list, year_list, Winter_24, Winter_25, Summer_24 = SOFiA.get_corrected_and_masked_variables_C0_C2_all_own_ssha(ds, var_type, correction_type, mask_type, smoothing)
    ''' STANDARD DEVIATION OF VARIABLE : '''
    std = np.nanstd(masked_d, axis=0)
    return std, lat, lon
'''  3) Mean (temporal over cycles) of ssh_karin(_2) removed geoid and tides and dac --> MDT '''     # DIMS: (num_lines,num_pixels)
def temporal_nanmean(ds, var_type, correction_type, mask_type, smoothing):
    #ds = xr.open_dataset(path_merged+pass_nr+'.nc')
    #masked_d, lat, lon, month_list, year_list, Winter_24, Winter_25, Summer_24 = SOFiA.get_corrected_and_masked_variables2(ds, var_type, correction_type, mask_type, smoothing)
    masked_d, lat, lon, month_list, year_list, Winter_24, Winter_25, Summer_24 = SOFiA.get_corrected_and_masked_variables_C0_C2_all_own_ssha(ds, var_type, correction_type, mask_type, smoothing)
    ''' MEAN OF VARIABLE : '''
    mean = np.nanmean(masked_d, axis=0)
    return mean, lat, lon
'''  4) Mean of slope (dSSH/dx) of MDT (over) --> Magnitude = sqrt(x² + y²) '''                      # DIMS: (num_lines,num_pixels)
def mean_slope_ssh(ds, var_type, correction_type, mask_type, smoothing):
    #ds = xr.open_dataset(path_merged+pass_nr+'.nc')
    #masked_d, lat, lon, month_list, year_list, Winter_24, Winter_25, Summer_24 = SOFiA.get_corrected_and_masked_variables2(ds, var_type, correction_type, mask_type, smoothing)
    masked_d, lat, lon, month_list, year_list, Winter_24, Winter_25, Summer_24 = SOFiA.get_corrected_and_masked_variables_C0_C2_all_own_ssha(ds, var_type, correction_type, mask_type, smoothing)
    ''' GET SLA_DX AND SLA_DY : '''
    sla_diff_x = np.gradient(masked_d, axis=2)
    sla_diff_y = np.gradient(masked_d, axis=1)
    ''' dx, dy : '''
    dx = 2000  # m
    dy = 2000  # m 
    ''' SLOPE : '''
    slope_x = (sla_diff_x / dx)
    slope_y = (sla_diff_y / dy)
    ''' MAGNITUDE OF SLOPE : '''
    magnitude = np.sqrt(slope_x**2 + slope_y**2)
    ''' MEAN OF CYCLES : '''
    mean_magnitude = np.nanmean(magnitude, axis=0)
    ''' NORMALIZED ? '''
    # Make bigger to avoid inf : 
    #mean_to_norm = mean_magnitude*1e6
    #mean_magnitude_norm = (mean_to_norm-np.nanmin(mean_to_norm))/(np.nanmax(mean_to_norm)-np.nanmin(mean_to_norm))
    return mean_magnitude, lat, lon
'''  5) Mean of slope (dH/dx) of depth_or_elevation --> Magnitude = sqrt(x² + y²) '''                # DIMS: (num_lines,num_pixels)
def mean_slope_bath(ds):
    #ds = xr.open_dataset(path_merged+pass_nr+'.nc')
    ''' Mask only Ocean data : '''
    good_data = ds['ancillary_surface_classification_flag'] > 1
    # GET variable : 
    variable = ds['depth_or_elevation']
    ''' GET SLA_DX AND SLA_DY : '''
    sla_diff_x = np.gradient(variable, axis=2)
    sla_diff_y = np.gradient(variable, axis=1)
    ''' dx, dy : '''
    dx = 2000  # m
    dy = 2000  # m 
    ''' SLOPE : '''
    slope_x = (sla_diff_x / dx)
    slope_y = (sla_diff_y / dy)
    ''' MAGNITUDE OF SLOPE . '''
    magnitude = np.sqrt(slope_x**2 + slope_y**2)
    # Apply the mask :
    masked_1 = np.ma.masked_where(good_data, magnitude)
    masked   = np.ma.filled(masked_1, np.nan)
    ''' MEAN OF CYCLES : '''
    mean_magnitude = np.nanmean(masked, axis=0)
    ''' NORMALIZED ? '''
    mean_magnitude_norm = (mean_magnitude-np.nanmin(mean_magnitude))/(np.nanmax(mean_magnitude)-np.nanmin(mean_magnitude))
    return mean_magnitude, mean_magnitude_norm

#ds.mean_dynamic_topography[0]

In [ ]:
''' CREATE VARIABLES PART 1 : '''
def part_1_variables(ds):
    count_ssh_karin, lat, lon               = count_no_nan(ds,          var_type='ssh_karin',   correction_type='ssh',      mask_type='all', smoothing=None)
    count_ssh_karin_2, lat, lon             = count_no_nan(ds,          var_type='ssh_karin_2', correction_type='ssh',      mask_type='all', smoothing=None)
    std_ssh_karin_2, lat, lon               = standard_deviation(ds,    var_type='ssh_karin_2', correction_type='ssh',      mask_type='all', smoothing=None)
    mean_ssh_karin_2_tides, lat, lon        = temporal_nanmean(ds,      var_type='ssh_karin_2', correction_type='ssh_tide', mask_type='all', smoothing=None)
    mean_slope_ssh_karin_2_tides, lat, lon  = mean_slope_ssh(ds,        var_type='ssh_karin_2', correction_type='ssh_tide', mask_type='all', smoothing=None)
    mean_slope_bathy, mean_slope_bathy_norm = mean_slope_bath(ds)
    mean_dynamic_topography_model = ds.mean_dynamic_topography[0]
    ''' Fix the Longitude : '''
    new_lon = np.where(lon > 180, lon - 360, lon)
    return lat, new_lon, count_ssh_karin, count_ssh_karin_2, std_ssh_karin_2, mean_ssh_karin_2_tides, mean_slope_ssh_karin_2_tides, mean_slope_bathy, mean_slope_bathy_norm, mean_dynamic_topography_model

In [ ]:
def part_1_NetCDF(path_merged,pass_nr):
    ''' OPEN DS : '''
    ds = xr.open_dataset(path_merged+pass_nr+'.nc')
    ''' GET VARIABLES : '''
    lat, new_lon, count_ssh_karin, count_ssh_karin_2, std_ssh_karin_2, mean_ssh_karin_2_tides, mean_slope_ssh_karin_2_tides, mean_slope_bathy, mean_slope_bathy_norm, mean_dynamic_topography_model = part_1_variables(ds)
    ''' Num_lines, Num_pixels '''
    num_lines  = lat.shape[0]
    num_pixels = lat.shape[1]
    ''' CREATE DataArrays '''
    count1   = xr.DataArray(count_ssh_karin,               dims=('num_lines', 'num_pixels'), coords={'num_lines': np.arange(num_lines), 'num_pixels': np.arange(num_pixels)})
    count2   = xr.DataArray(count_ssh_karin_2,             dims=('num_lines', 'num_pixels'), coords={'num_lines': np.arange(num_lines), 'num_pixels': np.arange(num_pixels)})
    std2     = xr.DataArray(std_ssh_karin_2,               dims=('num_lines', 'num_pixels'), coords={'num_lines': np.arange(num_lines), 'num_pixels': np.arange(num_pixels)})
    mean2    = xr.DataArray(mean_ssh_karin_2_tides,        dims=('num_lines', 'num_pixels'), coords={'num_lines': np.arange(num_lines), 'num_pixels': np.arange(num_pixels)})
    slope2   = xr.DataArray(mean_slope_ssh_karin_2_tides,  dims=('num_lines', 'num_pixels'), coords={'num_lines': np.arange(num_lines), 'num_pixels': np.arange(num_pixels)})
    slopeb   = xr.DataArray(mean_slope_bathy,              dims=('num_lines', 'num_pixels'), coords={'num_lines': np.arange(num_lines), 'num_pixels': np.arange(num_pixels)})
    slopeb2  = xr.DataArray(mean_slope_bathy_norm,         dims=('num_lines', 'num_pixels'), coords={'num_lines': np.arange(num_lines), 'num_pixels': np.arange(num_pixels)})
    mdt      = xr.DataArray(mean_dynamic_topography_model, dims=('num_lines', 'num_pixels'), coords={'num_lines': np.arange(num_lines), 'num_pixels': np.arange(num_pixels)})
    latc     = xr.DataArray(lat,                           dims=('num_lines', 'num_pixels'), coords={'num_lines': np.arange(num_lines), 'num_pixels': np.arange(num_pixels)})
    new_lonc = xr.DataArray(new_lon,                       dims=('num_lines', 'num_pixels'), coords={'num_lines': np.arange(num_lines), 'num_pixels': np.arange(num_pixels)})
    ''' CREATE DataSet '''
    dataset  = xr.Dataset({'count_ssh_karin'               : count1, 
                           'count_ssh_karin_2'             : count2, 
                           'std_ssh_karin_2'               : std2, 
                           'mean_ssh_karin_2_tides'        : mean2, 
                           'mean_slope_ssh_karin_2_tides'  : slope2, 
                           'mean_slope_bathy'              : slopeb, 
                           'mean_slope_bathy_norm'         : slopeb2, 
                           'mean_dynamic_topography_model' : mdt,
                           'latitude'                      : latc, 
                           'longitude'                     : new_lonc})
    ''' RETURN THE DataSet '''
    return dataset

#### Test :

In [ ]:
# PATH : 
path_merged = '/uio/hypatia/geofag-personlig/students/metos/ingvilob/SWOT_1_RAW_MERGED/version_2/'
pass_nr = 'pass_003'

In [ ]:
ds = xr.open_dataset(path_merged+pass_nr+'.nc')

In [ ]:
ds.concat_dim

In [ ]:
dataset1 = part_1_NetCDF(path_merged,pass_nr)
dataset1

In [ ]:
plt.imshow(dataset1.mean_dynamic_topography_model, vmin=-0.1, vmax=0.1)
plt.ylim(500)
plt.colorbar()

In [ ]:
fig, axs = plt.subplots(3, 3, figsize=(10, 10))
a=axs[0, 0].imshow(dataset1.count_ssh_karin)
axs[0, 0].set_title('count_ssh_karin')
axs[0, 0].set_ylim(200)
plt.colorbar(a)

b=axs[0, 1].imshow(dataset1.count_ssh_karin_2)
axs[0, 1].set_title('count_ssh_karin_2')
axs[0, 1].set_ylim(200)
plt.colorbar(b)

c=axs[0, 2].imshow(dataset1.std_ssh_karin_2, vmin=0.5, vmax=0.8)
axs[0, 2].set_title('std_ssh_karin_2')
axs[0, 2].set_ylim(200)
plt.colorbar(c)

d=axs[1, 0].imshow(dataset1.mean_ssh_karin_2_tides, vmin=-0.3, vmax=0.3, cmap=cmocean.cm.balance)
axs[1, 0].set_title('mean_ssh_karin_2_tides')
axs[1, 0].set_ylim(200)
plt.colorbar(d)

e=axs[1, 1].imshow(dataset1.mean_slope_ssh_karin_2_tides, vmin=0, vmax=0.00001, cmap=cmocean.cm.balance)
axs[1, 1].set_title('mean_slope_ssh_karin_2_tides')
axs[1, 1].set_ylim(200)
plt.colorbar(e)

f=axs[1, 2].imshow(dataset1.mean_slope_bathy)
axs[1, 2].set_title('mean_slope_bathy')
axs[1, 2].set_ylim(200)
plt.colorbar(f)

g=axs[2, 0].imshow(dataset1.mean_slope_bathy_norm)
axs[2, 0].set_title('mean_slope_bathy_norm')
axs[2, 0].set_ylim(200)
plt.colorbar(g)

h=axs[2, 1].imshow(dataset1.latitude, vmin=45, vmax=50)
axs[2, 1].set_title('latitude')
axs[2, 1].set_ylim(200)
plt.colorbar(h)

i=axs[2, 2].imshow(dataset1.longitude, vmin=-20, vmax=-15)
axs[2, 2].set_title('longitude')
axs[2, 2].set_ylim(200)
plt.colorbar(i)

# Adjust the layout
plt.tight_layout()
plt.show()

### PART 2 : 

In [ ]:
''' FUNCTIONS PART 2 : '''
'''  6) Standard deviation of SSHA '''                                                               # DIMS: (num_lines,num_pixels)
def standard_deviation(ds, var_type, correction_type, mask_type, smoothing):
    #ds = xr.open_dataset(path_merged+pass_nr+'.nc')
    #masked_d, lat, lon, month_list, year_list, Winter_24, Winter_25, Summer_24 = SOFiA.get_corrected_and_masked_variables2(ds, var_type, correction_type, mask_type, smoothing)
    masked_d, lat, lon, month_list, year_list, Winter_24, Winter_25, Summer_24 = SOFiA.get_corrected_and_masked_variables_C0_C2_all_own_ssha(ds, var_type, correction_type, mask_type, smoothing)
    ''' STANDARD DEVIATION OF VARIABLE : '''
    std = np.nanstd(masked_d, axis=0)
    return std, lat, lon
'''  7) Mean (temporal over cycles) of ssha_karin_2 '''                                              # DIMS: (num_lines,num_pixels)
def temporal_nanmean(ds, var_type, correction_type, mask_type, smoothing):
    #ds = xr.open_dataset(path_merged+pass_nr+'.nc')
    #masked_d, lat, lon, month_list, year_list, Winter_24, Winter_25, Summer_24 = SOFiA.get_corrected_and_masked_variables2(ds, var_type, correction_type, mask_type, smoothing)
    masked_d, lat, lon, month_list, year_list, Winter_24, Winter_25, Summer_24 = SOFiA.get_corrected_and_masked_variables_C0_C2_all_own_ssha(ds, var_type, correction_type, mask_type, smoothing)
    ''' MEAN OF VARIABLE : '''
    mean = np.nanmean(masked_d, axis=0)
    return mean, lat, lon
#'''  8) mean_dynamic_topography '''                                                                  # DIMS: (num_lines,num_pixels)
#ds.mean_dynamic_topography[0]
'''  9) EKE removed mean (SSHA) '''                                                                  # DIMS: (num_lines,num_pixels)
def calculate_ug_vg(masked_dataset, lat, lon):
    ''' GET SLA_DX AND SLA_DY : '''
    sla_diff_x = np.gradient(masked_dataset, axis=2)
    sla_diff_y = np.gradient(masked_dataset, axis=1)
    ''' dx, dy : '''
    dx = 2000
    dy = 2000
    ''' Tetha, g, f : '''
    tetha = np.deg2rad(lat)
    g     = 9.81
    f     = 2 * 7.2921e-5 * np.sin(tetha)
    ''' ug, vg : '''
    ug  = - (g / f) * (sla_diff_y / dy)
    vg  =   (g / f) * (sla_diff_x / dx)
    ''' RETURN UG AND VG : '''
    return ug, vg
def eke_normal(ds, var_type, correction_type, mask_type, smoothing):
    #ds = xr.open_dataset(path_merged+pass_nr+'.nc')
    #masked_d, lat, lon, month_list, year_list, Winter_24, Winter_25, Summer_24 = SOFiA.get_corrected_and_masked_variables2(ds, var_type, correction_type, mask_type, smoothing)
    masked_d, lat, lon, month_list, year_list, Winter_24, Winter_25, Summer_24 = SOFiA.get_corrected_and_masked_variables_C0_C2_all_own_ssha(ds, var_type, correction_type, mask_type, smoothing)
    ''' CALCULATE UG AND VG : '''
    ug, vg = calculate_ug_vg(masked_d, lat, lon)
    ''' MEAN OF ug AND vg : '''
    ug_mean = np.nanmean(ug, axis=0)
    vg_mean = np.nanmean(vg, axis=0)
    ''' EKE : '''
    ''' METHOD 1 : REMOVE MEAN BEFORE SQUARING : '''
    # EKE Jonathan : EKE = 1/2*[<(u-<u>)^2>+<(v-<v>)^2>] 
    ug_diff_mean = ug - ug_mean
    vg_diff_mean = vg - vg_mean
    ug_diff_sq = ug_diff_mean**2
    vg_diff_sq = vg_diff_mean**2
    eke_jon = 0.5 * (np.nanmean(ug_diff_sq, axis=0) + np.nanmean(vg_diff_sq, axis=0))
    return eke_jon, lat, lon 
''' 10) EKE not removed mean (SSHA) '''                                                              # DIMS: (num_lines,num_pixels)
def eke_normal_not_rem_mean(ds, var_type, correction_type, mask_type, smoothing):
    #ds = xr.open_dataset(path_merged+pass_nr+'.nc')
    #masked_d, lat, lon, month_list, year_list, Winter_24, Winter_25, Summer_24 = SOFiA.get_corrected_and_masked_variables2(ds, var_type, correction_type, mask_type, smoothing)
    masked_d, lat, lon, month_list, year_list, Winter_24, Winter_25, Summer_24 = SOFiA.get_corrected_and_masked_variables_C0_C2_all_own_ssha(ds, var_type, correction_type, mask_type, smoothing)
    ''' CALCULATE UG AND VG : '''
    ug, vg = calculate_ug_vg(masked_d, lat, lon)
    ''' EKE : '''
    eke_not_removed   = 0.5 * (np.nanmean(ug**2, axis=0) + np.nanmean(vg**2, axis=0))
    return eke_not_removed, lat, lon 


In [ ]:
''' CREATE VARIABLES PART 2 : '''
def part_2_variables(ds):
    std_ssha_karin_2, lat, lon          = standard_deviation(ds,        var_type='ssh_karin_2', correction_type='ssha_own', mask_type='all', smoothing=2)
    mean_ssha_karin_2, lat, lon         = temporal_nanmean(ds,          var_type='ssh_karin_2', correction_type='ssha_own', mask_type='all', smoothing=2)
    eke_ssha_karin_2_normal, lat, lon   = eke_normal(ds,                var_type='ssh_karin_2', correction_type='ssha_own', mask_type='all', smoothing=2)
    eke_ssha_karin_2_option_2, lat, lon = eke_normal_not_rem_mean(ds,   var_type='ssh_karin_2', correction_type='ssha_own', mask_type='all', smoothing=2)
    ''' Fix the Longitude : '''
    new_lon = np.where(lon > 180, lon - 360, lon)
    return lat, new_lon, std_ssha_karin_2, mean_ssha_karin_2, eke_ssha_karin_2_normal, eke_ssha_karin_2_option_2

In [ ]:
def part_2_NetCDF(path_merged,pass_nr):
    ''' OPEN DS : '''
    ds = xr.open_dataset(path_merged+pass_nr+'.nc')
    ''' GET VARIABLES : '''
    lat, new_lon, std_ssha_karin_2, mean_ssha_karin_2, eke_ssha_karin_2_normal, eke_ssha_karin_2_option_2 = part_2_variables(ds)
    ''' Num_lines, Num_pixels '''
    num_lines  = lat.shape[0]
    num_pixels = lat.shape[1]
    ''' CREATE DataArrays '''
    std   = xr.DataArray(std_ssha_karin_2,               dims=('num_lines', 'num_pixels'), coords={'num_lines': np.arange(num_lines), 'num_pixels': np.arange(num_pixels)})
    mean   = xr.DataArray(mean_ssha_karin_2,             dims=('num_lines', 'num_pixels'), coords={'num_lines': np.arange(num_lines), 'num_pixels': np.arange(num_pixels)})
    eke1     = xr.DataArray(eke_ssha_karin_2_normal,               dims=('num_lines', 'num_pixels'), coords={'num_lines': np.arange(num_lines), 'num_pixels': np.arange(num_pixels)})
    eke2    = xr.DataArray(eke_ssha_karin_2_option_2,        dims=('num_lines', 'num_pixels'), coords={'num_lines': np.arange(num_lines), 'num_pixels': np.arange(num_pixels)})
    latc     = xr.DataArray(lat,                           dims=('num_lines', 'num_pixels'), coords={'num_lines': np.arange(num_lines), 'num_pixels': np.arange(num_pixels)})
    new_lonc = xr.DataArray(new_lon,                       dims=('num_lines', 'num_pixels'), coords={'num_lines': np.arange(num_lines), 'num_pixels': np.arange(num_pixels)})
    ''' CREATE DataSet '''
    dataset  = xr.Dataset({'std_ssha_karin_2'              : std, 
                           'mean_ssha_karin_2'             : mean, 
                           'eke_ssha_karin_2_normal'       : eke1, 
                           'eke_ssha_karin_2_option_2'     : eke2, 
                           'latitude'                      : latc, 
                           'longitude'                     : new_lonc})
    ''' RETURN THE DataSet '''
    return dataset

#### Test :

In [ ]:
# PATH : 
path_merged = '/uio/hypatia/geofag-personlig/students/metos/ingvilob/SWOT_1_RAW_MERGED/version_2/'
pass_nr = 'pass_003'

In [ ]:
dataset2 = part_2_NetCDF(path_merged,pass_nr)
dataset2

In [ ]:
fig, axs = plt.subplots(2, 3, figsize=(10, 10))
a=axs[0, 0].imshow(dataset2.std_ssha_karin_2, vmin=0, vmax=0.2)
axs[0, 0].set_title('std_ssha_karin_2')
axs[0, 0].set_ylim(200)
plt.colorbar(a)

b=axs[0, 1].imshow(dataset2.mean_ssha_karin_2, vmin=-0.3, vmax=0.3, cmap=cmocean.cm.balance)
axs[0, 1].set_title('mean_ssha_karin_2')
axs[0, 1].set_ylim(200)
plt.colorbar(b)

c=axs[0, 2].imshow(dataset2.eke_ssha_karin_2_normal, vmin=0, vmax=0.3, cmap=cmocean.cm.matter)
axs[0, 2].set_title('eke_ssha_karin_2_normal')
axs[0, 2].set_ylim(200)
plt.colorbar(c)

d=axs[1, 0].imshow(dataset2.eke_ssha_karin_2_option_2, vmin=0, vmax=0.3, cmap=cmocean.cm.matter)
axs[1, 0].set_title('eke_ssha_karin_2_option_2')
axs[1, 0].set_ylim(200)
plt.colorbar(d)

e=axs[1, 1].imshow(dataset2.latitude, vmin=45, vmax=50)
axs[1, 1].set_title('latitude')
axs[1, 1].set_ylim(200)
plt.colorbar(e)

f=axs[1, 2].imshow(dataset2.longitude, vmin=-20, vmax=-15)
axs[1, 2].set_title('longitude')
axs[1, 2].set_ylim(200)
plt.colorbar(f)

# Adjust the layout
plt.tight_layout()
plt.show()

### PART 3 : 

In [ ]:
''' FUNCTIONS PART 3 : '''
''' 11) Mean ssha Winter-Summer '''                                                                # DIMS: (num_lines,num_pixels)
def temporal_nanmean_season(masked_d, lat, lon, Winter_24, Summer_24):#ds, var_type, correction_type, mask_type, smoothing):
    #ds = xr.open_dataset(path_merged+pass_nr+'.nc')
    #masked_d, lat, lon, month_list, year_list, Winter_24, Winter_25, Summer_24 = SOFiA.get_corrected_and_masked_variables2(ds, var_type, correction_type, mask_type, smoothing)
    ''' GET WINTER 24 AND SUMMER 24 : '''
    w_24 = masked_d[Winter_24]
    s_24 = masked_d[Summer_24]
    ''' MEAN OF VARIABLE : '''
    mean_w_24 = np.nanmean(w_24, axis=0)
    mean_s_24 = np.nanmean(s_24, axis=0)
    ''' Winter-Summer : '''
    mean = mean_w_24-mean_s_24
    return mean, lat, lon
''' 12) ug and vg (rotated) Winter-Summer --> speed? '''                                             # DIMS: (num_lines,num_pixels)
def calculate_ug_vg(masked_dataset, lat, lon):
    ''' GET SLA_DX AND SLA_DY : '''
    sla_diff_x = np.gradient(masked_dataset, axis=2)
    sla_diff_y = np.gradient(masked_dataset, axis=1)
    ''' dx, dy : '''
    dx = 2000
    dy = 2000
    ''' Tetha, g, f : '''
    tetha = np.deg2rad(lat)
    g     = 9.81
    f     = 2 * 7.2921e-5 * np.sin(tetha)
    ''' ug, vg : '''
    ug  = - (g / f) * (sla_diff_y / dy)
    vg  =   (g / f) * (sla_diff_x / dx)
    ''' RETURN UG AND VG : '''
    return ug, vg
def rotate_dataset_original_grid_all_cycles(lat, lon, u, v):
    TETHA = lon
    PHI   = lat
    num_lines  = PHI.shape[0]
    num_pixels = PHI.shape[1]
    TETHA      = np.where(TETHA > 180, TETHA - 360, TETHA)
    # because num_lines is y and num_pixels is x
    delta_PHI        = np.diff(PHI,   axis=1)
    delta_TETHA      = np.diff(TETHA, axis=1)
    delta_PHI        = np.append(delta_PHI, np.zeros((num_lines, 1)), axis=1)
    delta_TETHA      = np.append(delta_TETHA, np.zeros((num_lines, 1)), axis=1)
    tetha_rad        = np.deg2rad(TETHA)
    phi_rad          = np.deg2rad(PHI)
    delta_tetha_rad  = np.deg2rad(delta_TETHA)
    delta_phi_rad    = np.deg2rad(delta_PHI)
    cos_PHI          = np.cos(phi_rad)
    curly_TETHA      = np.arctan2(delta_phi_rad,cos_PHI*delta_tetha_rad)
    u_rotated        = u*np.cos(curly_TETHA) - v*np.sin(curly_TETHA)
    v_rotated        = u*np.sin(curly_TETHA) + v*np.cos(curly_TETHA)
    u_rotated_by_phi = u_rotated/cos_PHI
    return u_rotated, v_rotated
def ug_vg_season(masked_d, lat, lon, Winter_24, Summer_24):#ds, var_type, correction_type, mask_type, smoothing):
    #ds = xr.open_dataset(path_merged+pass_nr+'.nc')
    #masked_d, lat, lon, month_list, year_list, Winter_24, Winter_25, Summer_24 = SOFiA.get_corrected_and_masked_variables2(ds, var_type, correction_type, mask_type, smoothing)
    w_24 = masked_d[Winter_24]
    s_24 = masked_d[Summer_24]
    ''' MEAN OF VARIABLE : '''
    mean_w_24 = np.nanmean(w_24, axis=0)
    mean_s_24 = np.nanmean(s_24, axis=0)
    ''' Winter-Summer : '''
    mean = mean_w_24-mean_s_24
    ''' CALCULATE UG AND VG : '''
    sla_diff_x = np.gradient(mean, axis=1)
    sla_diff_y = np.gradient(mean, axis=0)
    ''' dx, dy : '''
    dx = 2000
    dy = 2000
    ''' Tetha, g, f : '''
    tetha = np.deg2rad(lat)
    g     = 9.81
    f     = 2 * 7.2921e-5 * np.sin(tetha)
    ''' ug, vg : '''
    ug_w_s  = - (g / f) * (sla_diff_y / dy)
    vg_w_s  =   (g / f) * (sla_diff_x / dx)
    ''' CALCULATE UG AND VG : '''
    ug, vg = calculate_ug_vg(masked_d, lat, lon)
    ''' GET THE SEASONAL INDICES : '''
    ug_w = ug[Winter_24]
    vg_w = vg[Winter_24]
    ug_s = ug[Summer_24]
    vg_s = vg[Summer_24]
    ''' MEAN OF ug AND vg  SEASONAL : '''
    ug_w_mean = np.nanmean(ug_w, axis=0)
    vg_w_mean = np.nanmean(vg_w, axis=0)
    ug_s_mean = np.nanmean(ug_s, axis=0)
    vg_s_mean = np.nanmean(vg_s, axis=0)
    ''' SPEED : '''
    speed_w = np.sqrt(ug_w_mean**2 + vg_w_mean**2)
    speed_s = np.sqrt(ug_s_mean**2 + vg_s_mean**2)
    ''' Winter-Summer : '''
    #ug_w_s = ug_w_mean-ug_s_mean
    #vg_w_s = vg_w_mean-vg_s_mean
    ''' ROTATE THEM : '''
    ug_w_s_r, vg_w_s_r = rotate_dataset_original_grid_all_cycles(lat, lon, ug_w_s, vg_w_s)
    ''' SPEED : '''
    #speed = np.sqrt(ug_w_s_r**2 + vg_w_s_r**2)
    speed = speed_w-speed_s
    return speed, ug_w_s_r, vg_w_s_r, lat, lon
''' 13) EKE Winter-Summer '''                                                                        # DIMS: (num_lines,num_pixels)
def eke_season(masked_d, lat, lon, Winter_24, Summer_24):#ds, var_type, correction_type, mask_type, smoothing):
    #ds = xr.open_dataset(path_merged+pass_nr+'.nc')
    #masked_d, lat, lon, month_list, year_list, Winter_24, Winter_25, Summer_24 = SOFiA.get_corrected_and_masked_variables2(ds, var_type, correction_type, mask_type, smoothing)
    ''' CALCULATE UG AND VG : '''
    ug, vg = calculate_ug_vg(masked_d, lat, lon)
    ''' GET THE SEASONAL INDICES : '''
    ug_w = ug[Winter_24]
    vg_w = vg[Winter_24]
    ug_s = ug[Summer_24]
    vg_s = vg[Summer_24]
    ''' MEAN OF ug AND vg  SEASONAL : '''
    ug_w_mean = np.nanmean(ug_w, axis=0)
    vg_w_mean = np.nanmean(vg_w, axis=0)
    ug_s_mean = np.nanmean(ug_s, axis=0)
    vg_s_mean = np.nanmean(vg_s, axis=0)
    ''' EKE : '''
    # WINTER : 
    ug_diff_mean_w = ug_w - ug_w_mean
    vg_diff_mean_w = vg_w - vg_w_mean
    ug_diff_sq_w = ug_diff_mean_w**2
    vg_diff_sq_w = vg_diff_mean_w**2
    eke_jon_w = 0.5 * (np.nanmean(ug_diff_sq_w, axis=0) + np.nanmean(vg_diff_sq_w, axis=0))
    # SUMMER : 
    ug_diff_mean_s = ug_s - ug_s_mean
    vg_diff_mean_s = vg_s - vg_s_mean
    ug_diff_sq_s = ug_diff_mean_s**2
    vg_diff_sq_s = vg_diff_mean_s**2
    eke_jon_s = 0.5 * (np.nanmean(ug_diff_sq_s, axis=0) + np.nanmean(vg_diff_sq_s, axis=0))
    ''' Winter-Summer : '''
    eke_seasonal = eke_jon_w-eke_jon_s
    return eke_seasonal, lat, lon


In [ ]:
''' CREATE VARIABLES PART 3 : '''
def part_3_variables(ds):
    #masked_d, lat, lon, month_list, year_list, Winter_24, Winter_25, Summer_24 = SOFiA.get_corrected_and_masked_variables2(ds, var_type='ssha_karin_2', correction_type='ssha', mask_type='all', smoothing=None)
    #masked_d, lat, lon, month_list, year_list, Winter_24, Winter_25, Summer_24 = SOFiA.get_corrected_and_masked_variables2(ds, var_type='ssha_karin_2', correction_type='ssha', mask_type='all', smoothing=1)
    masked_d, lat, lon, month_list, year_list, Winter_24, Winter_25, Summer_24 = SOFiA.get_corrected_and_masked_variables_C0_C2_all_own_ssha(ds, var_type='ssh_karin_2', correction_type='ssha_own', mask_type='all', smoothing=2)
    mean_ssha_karin_2_W_S, lat, lon                      = temporal_nanmean_season(masked_d, lat, lon, Winter_24, Summer_24)#ds,  var_type='ssha_karin_2', correction_type='ssha', mask_type='all', smoothing=None)
    speed_ssha_karin_2_W_S, ug_w_s_r, vg_w_s_r, lat, lon = ug_vg_season(masked_d, lat, lon, Winter_24, Summer_24)#ds,             var_type='ssha_karin_2', correction_type='ssha', mask_type='all', smoothing=None)
    eke_ssha_karin_2_W_S, lat, lon                       = eke_season(masked_d, lat, lon, Winter_24, Summer_24)#ds,               var_type='ssha_karin_2', correction_type='ssha', mask_type='all', smoothing=None)
    ''' Fix the Longitude : '''
    new_lon = np.where(lon > 180, lon - 360, lon)
    return lat, new_lon, mean_ssha_karin_2_W_S, speed_ssha_karin_2_W_S, ug_w_s_r, vg_w_s_r, eke_ssha_karin_2_W_S

In [ ]:
def part_3_NetCDF(path_merged,pass_nr):
    ''' OPEN DS : '''
    ds = xr.open_dataset(path_merged+pass_nr+'.nc')
    ''' GET VARIABLES : '''
    lat, new_lon, mean_ssha_karin_2_W_S, speed_ssha_karin_2_W_S, ug_w_s_r, vg_w_s_r, eke_ssha_karin_2_W_S = part_3_variables(ds)
    ''' Num_lines, Num_pixels '''
    num_lines  = lat.shape[0]
    num_pixels = lat.shape[1]
    ''' CREATE DataArrays '''
    mean     = xr.DataArray(mean_ssha_karin_2_W_S,  dims=('num_lines', 'num_pixels'), coords={'num_lines': np.arange(num_lines), 'num_pixels': np.arange(num_pixels)})
    speed    = xr.DataArray(speed_ssha_karin_2_W_S, dims=('num_lines', 'num_pixels'), coords={'num_lines': np.arange(num_lines), 'num_pixels': np.arange(num_pixels)})
    ug       = xr.DataArray(ug_w_s_r,               dims=('num_lines', 'num_pixels'), coords={'num_lines': np.arange(num_lines), 'num_pixels': np.arange(num_pixels)})
    vg       = xr.DataArray(vg_w_s_r,               dims=('num_lines', 'num_pixels'), coords={'num_lines': np.arange(num_lines), 'num_pixels': np.arange(num_pixels)})
    eke      = xr.DataArray(eke_ssha_karin_2_W_S,   dims=('num_lines', 'num_pixels'), coords={'num_lines': np.arange(num_lines), 'num_pixels': np.arange(num_pixels)})
    latc     = xr.DataArray(lat,                    dims=('num_lines', 'num_pixels'), coords={'num_lines': np.arange(num_lines), 'num_pixels': np.arange(num_pixels)})
    new_lonc = xr.DataArray(new_lon,                dims=('num_lines', 'num_pixels'), coords={'num_lines': np.arange(num_lines), 'num_pixels': np.arange(num_pixels)})
    ''' CREATE DataSet '''
    dataset  = xr.Dataset({'mean_ssha_karin_2_W_S'  : mean, 
                           'speed_ssha_karin_2_W_S' : speed, 
                           'ug_w_s_r'               : ug, 
                           'vg_w_s_r'               : vg, 
                           'eke_ssha_karin_2_W_S'   : eke, 
                           'latitude'               : latc, 
                           'longitude'              : new_lonc})
    ''' RETURN THE DataSet '''
    return dataset

#### Test :

In [ ]:
# PATH : 
path_merged = '/uio/hypatia/geofag-personlig/students/metos/ingvilob/SWOT_1_RAW_MERGED/version_2/'
pass_nr = 'pass_003'

In [ ]:
dataset3 = part_3_NetCDF(path_merged,pass_nr)
dataset3

In [ ]:
fig, axs = plt.subplots(2, 3, figsize=(10, 10))
a=axs[0, 0].imshow(dataset3.mean_ssha_karin_2_W_S, vmin=-0.2, vmax=0.2, cmap=cmocean.cm.balance)
axs[0, 0].set_title('mean_ssha_karin_2_W_S')
axs[0, 0].set_ylim(200)
plt.colorbar(a)

b=axs[0, 1].imshow(dataset3.speed_ssha_karin_2_W_S, vmin=0, vmax=0.5, cmap=cmocean.cm.speed)
axs[0, 1].set_title('speed_ssha_karin_2_W_S')
axs[0, 1].set_ylim(200)
plt.colorbar(b)

c=axs[0, 2].imshow(dataset3.ug_w_s_r, vmin=-0.3, vmax=0.3, cmap=cmocean.cm.matter)
axs[0, 2].set_title('ug_w_s_r')
axs[0, 2].set_ylim(200)
plt.colorbar(c)

d=axs[1, 0].imshow(dataset3.vg_w_s_r, vmin=-0.3, vmax=0.3, cmap=cmocean.cm.balance)
axs[1, 0].set_title('vg_w_s_r')
axs[1, 0].set_ylim(200)
plt.colorbar(d)

e=axs[1, 1].imshow(dataset3.eke_ssha_karin_2_W_S, vmin=0, vmax=0.5, cmap=cmocean.cm.matter)
axs[1, 1].set_title('eke_ssha_karin_2_W_S')
axs[1, 1].set_ylim(200)
plt.colorbar(e)

f=axs[1, 2].imshow(dataset3.longitude, vmin=-20, vmax=-15)
axs[1, 2].set_title('longitude')
axs[1, 2].set_ylim(200)
plt.colorbar(f)

# Adjust the layout
plt.tight_layout()
plt.show()

In [ ]:
save_path = '/uio/hypatia/geofag-personlig/students/metos/ingvilob/Investigation_part_1/'
dataset_part_3 = part_3_NetCDF(path_merged,pass_nr)
dataset_part_3.to_netcdf(save_path+'testpart_3_'+pass_nr+'.nc')
print(save_path+'part_3_'+pass_nr+'.nc')

### PART 4 : 

In [ ]:
''' FUNCTIONS PART 4 : '''
def calculate_ug_vg(masked_dataset, lat, lon):
    ''' GET SLA_DX AND SLA_DY : '''
    sla_diff_x = np.gradient(masked_dataset, axis=2)
    sla_diff_y = np.gradient(masked_dataset, axis=1)
    ''' dx, dy : '''
    dx = 2000
    dy = 2000
    ''' Tetha, g, f : '''
    tetha = np.deg2rad(lat)
    g     = 9.81
    f     = 2 * 7.2921e-5 * np.sin(tetha)
    ''' ug, vg : '''
    ug  = - (g / f) * (sla_diff_y / dy)
    vg  =   (g / f) * (sla_diff_x / dx)
    ''' RETURN UG AND VG : '''
    return ug, vg
def rotate_dataset_original_grid_all_cycles(lat, lon, u, v):
    TETHA = lon
    PHI   = lat
    num_lines  = PHI.shape[0]
    num_pixels = PHI.shape[1]
    TETHA      = np.where(TETHA > 180, TETHA - 360, TETHA)
    # because num_lines is y and num_pixels is x
    delta_PHI        = np.diff(PHI,   axis=1)
    delta_TETHA      = np.diff(TETHA, axis=1)
    delta_PHI        = np.append(delta_PHI, np.zeros((num_lines, 1)), axis=1)
    delta_TETHA      = np.append(delta_TETHA, np.zeros((num_lines, 1)), axis=1)
    tetha_rad        = np.deg2rad(TETHA)
    phi_rad          = np.deg2rad(PHI)
    delta_tetha_rad  = np.deg2rad(delta_TETHA)
    delta_phi_rad    = np.deg2rad(delta_PHI)
    cos_PHI          = np.cos(phi_rad)
    curly_TETHA      = np.arctan2(delta_phi_rad,cos_PHI*delta_tetha_rad)
    u_rotated        = u*np.cos(curly_TETHA) - v*np.sin(curly_TETHA)
    v_rotated        = u*np.sin(curly_TETHA) + v*np.cos(curly_TETHA)
    u_rotated_by_phi = u_rotated/cos_PHI
    return u_rotated, v_rotated
''' STANDARD DEVIATION OF UG AND VG - TRACK : '''
def STD_ug_vg_track(masked_d, lat, lon):
    ''' CALCULATE UG AND VG : '''
    ug, vg = calculate_ug_vg(masked_d, lat, lon)
    ''' MEAN OF ug AND vg : '''
    #ug_mean = np.nanmean(ug, axis=0)
    #vg_mean = np.nanmean(vg, axis=0)
    ''' STD : '''
    std_ug = np.nanstd(ug, axis=0)
    std_vg = np.nanstd(vg, axis=0)
    return std_ug, std_vg
''' STANDARD DEVIATION OF UG AND VG - ZONAL AND MERIDIONAL : '''
def STD_ug_vg_zon_mer(masked_d, lat, lon):
    ''' CALCULATE UG AND VG : '''
    ug, vg = calculate_ug_vg(masked_d, lat, lon)
    ''' MEAN OF ug AND vg : '''
    #ug_mean = np.nanmean(ug, axis=0)
    #vg_mean = np.nanmean(vg, axis=0)
    ''' ROTATE TO ZONAL AND MERIDIONAL : '''
    ug_rot, vg_rot = rotate_dataset_original_grid_all_cycles(lat, lon, ug, vg)
    ''' STD : '''
    std_ug = np.nanstd(ug_rot, axis=0)
    std_vg = np.nanstd(vg_rot, axis=0)
    return std_ug, std_vg

In [ ]:
''' CREATE VARIABLES PART 4 : '''
def part_4_variables(ds):
    #masked_d, lat, lon, month_list, year_list, Winter_24, Winter_25, Summer_24 = SOFiA.get_corrected_and_masked_variables2(ds, var_type='ssha_karin_2', correction_type='ssha', mask_type='all', smoothing=None)
    masked_d, lat, lon, month_list, year_list, Winter_24, Winter_25, Summer_24 = SOFiA.get_corrected_and_masked_variables_C0_C2_all_own_ssha(ds, var_type='ssh_karin_2', correction_type='ssha_own', mask_type='all', smoothing=None)
    #masked_d, lat, lon, month_list, year_list, Winter_24, Winter_25, Summer_24 = SOFiA.get_corrected_and_masked_variables2(ds, var_type='ssha_karin_2', correction_type='ssha', mask_type='all', smoothing=1)
    std_ug_T, std_vg_T = STD_ug_vg_track(masked_d, lat, lon)
    std_ug_G, std_vg_G = STD_ug_vg_zon_mer(masked_d, lat, lon)
    ''' Fix the Longitude : '''
    new_lon = np.where(lon > 180, lon - 360, lon)
    return lat, new_lon, std_ug_T, std_vg_T, std_ug_G, std_vg_G

In [ ]:
def part_4_NetCDF(path_merged,pass_nr):
    ''' OPEN DS : '''
    ds = xr.open_dataset(path_merged+pass_nr+'.nc')
    ''' GET VARIABLES : '''
    lat, new_lon, std_ug_T, std_vg_T, std_ug_G, std_vg_G = part_4_variables(ds)
    ''' Num_lines, Num_pixels '''
    num_lines  = lat.shape[0]
    num_pixels = lat.shape[1]
    ''' CREATE DataArrays '''
    stdugt   = xr.DataArray(std_ug_T, dims=('num_lines', 'num_pixels'), coords={'num_lines': np.arange(num_lines), 'num_pixels': np.arange(num_pixels)})
    stdvgt   = xr.DataArray(std_vg_T, dims=('num_lines', 'num_pixels'), coords={'num_lines': np.arange(num_lines), 'num_pixels': np.arange(num_pixels)})
    stdugg   = xr.DataArray(std_ug_G, dims=('num_lines', 'num_pixels'), coords={'num_lines': np.arange(num_lines), 'num_pixels': np.arange(num_pixels)})
    stdvgg   = xr.DataArray(std_vg_G, dims=('num_lines', 'num_pixels'), coords={'num_lines': np.arange(num_lines), 'num_pixels': np.arange(num_pixels)})
    latc     = xr.DataArray(lat,      dims=('num_lines', 'num_pixels'), coords={'num_lines': np.arange(num_lines), 'num_pixels': np.arange(num_pixels)})
    new_lonc = xr.DataArray(new_lon,  dims=('num_lines', 'num_pixels'), coords={'num_lines': np.arange(num_lines), 'num_pixels': np.arange(num_pixels)})
    ''' CREATE DataSet '''
    dataset  = xr.Dataset({'std_ug_zomer' : stdugg, 
                           'std_vg_zomer' : stdvgg, 
                           'std_ug_track' : stdugt, 
                           'std_vg_track' : stdvgt, 
                           'latitude'     : latc, 
                           'longitude'    : new_lonc})
    ''' RETURN THE DataSet '''
    return dataset

#### Testing : 

In [ ]:
# PATH : 
path_merged = '/uio/hypatia/geofag-personlig/students/metos/ingvilob/SWOT_1_RAW_MERGED/version_2/'
pass_nr = 'pass_003'

In [ ]:
dataset = part_4_NetCDF(path_merged,pass_nr)

In [ ]:
dataset

In [ ]:
#plt.imshow(dataset.std_ug_zomer,vmin=0,vmax=0.5)
#plt.ylim(250,500)
#plt.colorbar()
#plt.show()
plt.imshow(dataset.std_vg_zomer,vmin=0,vmax=0.5)
plt.ylim(400,500)
plt.colorbar()
plt.show()
#plt.imshow(dataset.std_ug_track,vmin=0,vmax=0.5)
#plt.ylim(250,500)
#plt.colorbar()
#plt.show()
plt.imshow(dataset.std_vg_track,vmin=0,vmax=0.5)
plt.ylim(400,500)
plt.colorbar()
plt.show()
plt.imshow(dataset.std_vg_zomer-dataset.std_vg_track,cmap=cmocean.cm.balance,vmin=-0.2,vmax=0.2)
plt.ylim(400,500)
plt.colorbar()
plt.show()

### PART 5 : 

In [ ]:
''' FUNCTIONS PART 5 : '''
def calculate_ug_vg(masked_dataset, lat, lon):
    ''' GET SLA_DX AND SLA_DY : '''
    sla_diff_x = np.gradient(masked_dataset, axis=2)
    sla_diff_y = np.gradient(masked_dataset, axis=1)
    ''' dx, dy : '''
    dx = 2000
    dy = 2000
    ''' Tetha, g, f : '''
    tetha = np.deg2rad(lat)
    g     = 9.81
    f     = 2 * 7.2921e-5 * np.sin(tetha)
    ''' ug, vg : '''
    ug  = - (g / f) * (sla_diff_y / dy)
    vg  =   (g / f) * (sla_diff_x / dx)
    ''' RETURN UG AND VG : '''
    return ug, vg
def rotate_dataset_original_grid_all_cycles(lat, lon, u, v):
    TETHA = lon
    PHI   = lat
    num_lines  = PHI.shape[0]
    num_pixels = PHI.shape[1]
    TETHA      = np.where(TETHA > 180, TETHA - 360, TETHA)
    # because num_lines is y and num_pixels is x
    delta_PHI        = np.diff(PHI,   axis=1)
    delta_TETHA      = np.diff(TETHA, axis=1)
    delta_PHI        = np.append(delta_PHI, np.zeros((num_lines, 1)), axis=1)
    delta_TETHA      = np.append(delta_TETHA, np.zeros((num_lines, 1)), axis=1)
    tetha_rad        = np.deg2rad(TETHA)
    phi_rad          = np.deg2rad(PHI)
    delta_tetha_rad  = np.deg2rad(delta_TETHA)
    delta_phi_rad    = np.deg2rad(delta_PHI)
    cos_PHI          = np.cos(phi_rad)
    curly_TETHA      = np.arctan2(delta_phi_rad,cos_PHI*delta_tetha_rad)
    u_rotated        = u*np.cos(curly_TETHA) - v*np.sin(curly_TETHA)
    v_rotated        = u*np.sin(curly_TETHA) + v*np.cos(curly_TETHA)
    u_rotated_by_phi = u_rotated/cos_PHI
    return u_rotated, v_rotated
''' Coefficient of Variation : '''
def CV_ug_vg_zon_mer(masked_d, lat, lon):
    ''' CALCULATE UG AND VG : '''
    ug, vg = calculate_ug_vg(masked_d, lat, lon)
    ''' ROTATE TO ZONAL AND MERIDIONAL : '''
    ug_rot, vg_rot = rotate_dataset_original_grid_all_cycles(lat, lon, ug, vg)
    ''' MEAN OF ug AND vg : '''
    ug_mean = np.nanmean(ug_rot, axis=0)
    vg_mean = np.nanmean(vg_rot, axis=0)
    ''' STD : '''
    std_ug = np.nanstd(ug_rot, axis=0)
    std_vg = np.nanstd(vg_rot, axis=0)
    ''' CV : '''
    CV_ug = (std_ug+1)/(np.abs(ug_mean)+1)
    CV_vg = (std_vg+1)/(np.abs(vg_mean)+1)
    return CV_ug, CV_vg, ug_mean, vg_mean, std_ug, std_vg#np.sqrt(CV_ug), np.sqrt(CV_vg), ug_mean, vg_mean#CV_ug/20, CV_vg/20#

def CV_speed_func(masked_d, lat, lon):
    ''' CALCULATE UG AND VG : '''
    ug, vg = calculate_ug_vg(masked_d, lat, lon)
    ''' ROTATE TO ZONAL AND MERIDIONAL : '''
    ug_rot, vg_rot = rotate_dataset_original_grid_all_cycles(lat, lon, ug, vg)
    ''' MEAN OF ug AND vg : '''
    ug_mean = np.nanmean(ug_rot, axis=0)
    vg_mean = np.nanmean(vg_rot, axis=0)
    ''' SPEED : '''
    speed = np.sqrt(ug_rot**2 + vg_rot**2)
    #mspeed = np.sqrt(ug_mean**2 + vg_mean**2)
    m2speed = np.nanmean(speed, axis=0)
    ''' STD : '''
    #std_ug = np.nanstd(ug_rot, axis=0)
    #std_vg = np.nanstd(vg_rot, axis=0)
    std_speed = np.nanstd(speed, axis=0)
    ''' CV : '''
    #CV_ug = (std_ug+1)/(np.abs(ug_mean)+1)
    #CV_vg = (std_vg+1)/(np.abs(vg_mean)+1)
    CV_speed = std_speed/m2speed
    #ug_diff_mean = ug - ug_mean
    #vg_diff_mean = vg - vg_mean
    #ug_diff_sq = ug_diff_mean**2
    #vg_diff_sq = vg_diff_mean**2
    #eke_jon = 0.5 * (np.nanmean(ug_diff_sq, axis=0) + np.nanmean(vg_diff_sq, axis=0))
    return  m2speed, std_speed, CV_speed


''' Spatial Standard deviation : '''

In [ ]:
''' CREATE VARIABLES PART 3 : '''
def part_5_variables(ds):
    #masked_d, lat, lon, month_list, year_list, Winter_24, Winter_25, Summer_24 = SOFiA.get_corrected_and_masked_variables2(ds, var_type='ssha_karin_2', correction_type='ssha', mask_type='all', smoothing=None)
    masked_d, lat, lon, month_list, year_list, Winter_24, Winter_25, Summer_24 = SOFiA.get_corrected_and_masked_variables_C0_C2_all_own_ssha(ds, var_type='ssh_karin_2', correction_type='ssha_own', mask_type='all', smoothing=None)
    #masked_d, lat, lon, month_list, year_list, Winter_24, Winter_25, Summer_24 = SOFiA.get_corrected_and_masked_variables2(ds, var_type='ssha_karin_2', correction_type='ssha', mask_type='all', smoothing=1)
    #CV_ug, CV_vg, ug_mean, vg_mean, std_ug, std_vg = CV_ug_vg_zon_mer(masked_d, lat, lon)
    m2speed, std_speed, CV_speed = CV_speed_func(masked_d, lat, lon)
    ''' Fix the Longitude : '''
    new_lon = np.where(lon > 180, lon - 360, lon)
    return lat, new_lon, m2speed, std_speed, CV_speed

In [ ]:
def part_5_NetCDF_old(path_merged,pass_nr):
    ''' OPEN DS : '''
    ds = xr.open_dataset(path_merged+pass_nr+'.nc')
    ''' GET VARIABLES : '''
    lat, new_lon, CV_ug, CV_vg, ug_mean, vg_mean, std_ug, std_vg = part_5_variables(ds)
    ''' Num_lines, Num_pixels '''
    num_lines  = lat.shape[0]
    num_pixels = lat.shape[1]
    ''' CREATE DataArrays '''
    CVug     = xr.DataArray(CV_ug, dims=('num_lines', 'num_pixels'), coords={'num_lines': np.arange(num_lines), 'num_pixels': np.arange(num_pixels)})
    CVvg     = xr.DataArray(CV_vg, dims=('num_lines', 'num_pixels'), coords={'num_lines': np.arange(num_lines), 'num_pixels': np.arange(num_pixels)})
    mug      = xr.DataArray(ug_mean, dims=('num_lines', 'num_pixels'), coords={'num_lines': np.arange(num_lines), 'num_pixels': np.arange(num_pixels)})
    mvg      = xr.DataArray(vg_mean, dims=('num_lines', 'num_pixels'), coords={'num_lines': np.arange(num_lines), 'num_pixels': np.arange(num_pixels)})
    latc     = xr.DataArray(lat,      dims=('num_lines', 'num_pixels'), coords={'num_lines': np.arange(num_lines), 'num_pixels': np.arange(num_pixels)})
    new_lonc = xr.DataArray(new_lon,  dims=('num_lines', 'num_pixels'), coords={'num_lines': np.arange(num_lines), 'num_pixels': np.arange(num_pixels)})
    ''' CREATE DataSet '''
    dataset  = xr.Dataset({'CV_ug'     : CVug, 
                           'CV_vg'     : CVvg, 
                           'ug_mean'   : mug, 
                           'vg_mean'   : mvg, 
                           'latitude'  : latc, 
                           'longitude' : new_lonc})
    ''' RETURN THE DataSet '''
    return dataset
def part_5_NetCDF(path_merged,pass_nr):
    ''' OPEN DS : '''
    ds = xr.open_dataset(path_merged+pass_nr+'.nc')
    ''' GET VARIABLES : '''
    lat, new_lon, m2speed, std_speed, CV_speed = part_5_variables(ds)
    ''' Num_lines, Num_pixels '''
    num_lines  = lat.shape[0]
    num_pixels = lat.shape[1]
    ''' CREATE DataArrays '''
    ms       = xr.DataArray(m2speed,   dims=('num_lines', 'num_pixels'), coords={'num_lines': np.arange(num_lines), 'num_pixels': np.arange(num_pixels)})
    stds     = xr.DataArray(std_speed, dims=('num_lines', 'num_pixels'), coords={'num_lines': np.arange(num_lines), 'num_pixels': np.arange(num_pixels)})
    cvs      = xr.DataArray(CV_speed,  dims=('num_lines', 'num_pixels'), coords={'num_lines': np.arange(num_lines), 'num_pixels': np.arange(num_pixels)})
    latc     = xr.DataArray(lat,       dims=('num_lines', 'num_pixels'), coords={'num_lines': np.arange(num_lines), 'num_pixels': np.arange(num_pixels)})
    new_lonc = xr.DataArray(new_lon,   dims=('num_lines', 'num_pixels'), coords={'num_lines': np.arange(num_lines), 'num_pixels': np.arange(num_pixels)})
    ''' CREATE DataSet '''
    dataset  = xr.Dataset({'mean_speed' : ms, 
                           'std_speed'  : stds, 
                           'CV_speed'   : cvs, 
                           'latitude'   : latc, 
                           'longitude'  : new_lonc})
    ''' RETURN THE DataSet '''
    return dataset

#### Testing : 

In [ ]:
# PATH : 
path_merged = '/uio/hypatia/geofag-personlig/students/metos/ingvilob/SWOT_1_RAW_MERGED/version_2/'
pass_nr = 'pass_003'

In [ ]:
dataset = part_5_NetCDF(path_merged,pass_nr)

In [ ]:
dataset

In [ ]:
plt.imshow(dataset.mean_speed,vmin=0,vmax=0.5)
plt.ylim(400,500)
plt.colorbar()
plt.show()
plt.imshow(dataset.std_speed,vmin=0,vmax=0.5)
plt.ylim(400,500)
plt.colorbar()
plt.show()
plt.imshow(dataset.CV_speed,vmin=0,vmax=1.5)
plt.ylim(400,500)
plt.colorbar()
plt.show()

## Make NetCDF-files : 

In [ ]:
passes = ['pass_003', 'pass_005', 'pass_007', 'pass_008', 'pass_009', 'pass_010', 'pass_012', 'pass_014', 'pass_016', 'pass_018',
          'pass_031', 'pass_033', 'pass_035', 'pass_036', 'pass_037', 'pass_038', 'pass_040', 'pass_042', 'pass_044', 'pass_046',
          'pass_059', 'pass_061', 'pass_063', 'pass_064', 'pass_065', 'pass_066', 'pass_068', 'pass_070', 'pass_072', 'pass_074',
          'pass_087', 'pass_089', 'pass_091', 'pass_092', 'pass_093', 'pass_094', 'pass_096', 'pass_098', 'pass_100',
          'pass_113', 'pass_115', 'pass_117', 'pass_119', 'pass_120', 'pass_121', 'pass_122', 'pass_124', 'pass_126', 'pass_128',
          'pass_141', 'pass_143', 'pass_145', 'pass_147', 'pass_148', 'pass_149', 'pass_150', 'pass_152', 'pass_154', 'pass_156',
          'pass_169', 'pass_171', 'pass_173', 'pass_175', 'pass_176', 'pass_177', 'pass_178', 'pass_180', 'pass_182', 'pass_184',
          'pass_197', 'pass_199', 'pass_201', 'pass_203', 'pass_204', 'pass_205', 'pass_206', 'pass_208', 'pass_210', 'pass_212',
          'pass_225', 'pass_227', 'pass_229', 'pass_231', 'pass_232', 'pass_233', 'pass_234', 'pass_236', 'pass_238', 'pass_240',
          'pass_253', 'pass_255', 'pass_257', 'pass_259', 'pass_260', 'pass_262', 'pass_264', 'pass_266', 'pass_268',
          'pass_281', 'pass_283', 'pass_285', 'pass_286', 'pass_287', 'pass_288', 'pass_290', 'pass_292', 'pass_294', 'pass_296',
          'pass_309', 'pass_311', 'pass_313', 'pass_314', 'pass_315', 'pass_316', 'pass_318', 'pass_320', 'pass_322', 'pass_324',
          'pass_337', 'pass_339', 'pass_341', 'pass_342', 'pass_343', 'pass_344', 'pass_346', 'pass_348', 'pass_350', 'pass_352',
          'pass_365', 'pass_367', 'pass_369', 'pass_370', 'pass_371', 'pass_372', 'pass_374', 'pass_376', 'pass_378', 'pass_391',
          'pass_393', 'pass_395', 'pass_397', 'pass_398', 'pass_399', 'pass_400', 'pass_402', 'pass_404', 'pass_406', 'pass_419',
          'pass_421', 'pass_423', 'pass_425', 'pass_426', 'pass_427', 'pass_428', 'pass_430', 'pass_432', 'pass_434', 'pass_447',
          'pass_449', 'pass_451', 'pass_453', 'pass_454', 'pass_455', 'pass_456', 'pass_458', 'pass_460', 'pass_462', 'pass_475',
          'pass_477', 'pass_479', 'pass_481', 'pass_482', 'pass_483', 'pass_484', 'pass_486', 'pass_488', 'pass_490', 'pass_503',
          'pass_505', 'pass_507', 'pass_509', 'pass_510', 'pass_511', 'pass_512', 'pass_514', 'pass_516', 'pass_518', 'pass_531',
          'pass_533', 'pass_535', 'pass_536', 'pass_537', 'pass_538', 'pass_539', 'pass_540', 'pass_542', 'pass_544', 'pass_546', 'pass_559',
          'pass_561', 'pass_563', 'pass_564', 'pass_565', 'pass_566', 'pass_568', 'pass_570', 'pass_572', 'pass_574']

In [ ]:
passes = ['pass_477', 'pass_479', 'pass_481', 'pass_482', 'pass_483', 'pass_484', 'pass_486', 'pass_488', 'pass_490', 'pass_503',
          'pass_505', 'pass_507', 'pass_509', 'pass_510', 'pass_511', 'pass_512', 'pass_514', 'pass_516', 'pass_518', 'pass_531',
          'pass_533', 'pass_535', 'pass_536', 'pass_537', 'pass_538', 'pass_539', 'pass_540', 'pass_542', 'pass_544', 'pass_546', 'pass_559',
          'pass_561', 'pass_563', 'pass_564', 'pass_565', 'pass_566', 'pass_568', 'pass_570', 'pass_572', 'pass_574']

In [ ]:
passes = ['pass_568', 'pass_570', 'pass_572', 'pass_574']

In [ ]:
save_path   = '/uio/hypatia/geofag-personlig/students/metos/ingvilob/Investigation_part_1/'
#save_path  = '/uio/hypatia/geofag-personlig/students/metos/ingvilob/Investigation_part_1/Smoothed_1_time/'
path_merged = '/uio/hypatia/geofag-personlig/students/metos/ingvilob/SWOT_1_RAW_MERGED/version_2/'

In [ ]:
for pass_nr in passes: 
    print(pass_nr)
    dataset_part_1 = part_1_NetCDF(path_merged,pass_nr)
    dataset_part_1.to_netcdf(save_path+'part_1/'+'part_1_'+pass_nr+'.nc')
    print(save_path+'part_1/'+'part_1_'+pass_nr+'.nc')

In [ ]:
for pass_nr in passes: 
    print(pass_nr)
    dataset_part_2 = part_2_NetCDF(path_merged,pass_nr)
    #dataset_part_2.to_netcdf(save_path+'part_2/'+'part_2_'+pass_nr+'.nc')
    dataset_part_2.to_netcdf(save_path+'part_2_sm_2/'+'part_2_'+pass_nr+'.nc')
    print(save_path+'part_2/'+'part_2_'+pass_nr+'.nc')

In [ ]:
for pass_nr in passes: 
    print(pass_nr)
    dataset_part_3 = part_3_NetCDF(path_merged,pass_nr)
    #dataset_part_3.to_netcdf(save_path+'part_3/'+'part_3_'+pass_nr+'.nc')
    dataset_part_3.to_netcdf(save_path+'part_3_sm_2/'+'part_3_'+pass_nr+'.nc')
    print(save_path+'part_3/'+'part_3_'+pass_nr+'.nc')

In [ ]:
for pass_nr in passes: 
    print(pass_nr)
    dataset_part_4 = part_4_NetCDF(path_merged,pass_nr)
    dataset_part_4.to_netcdf(save_path+'part_4/'+'part_4_'+pass_nr+'.nc')
    print(save_path+'part_4/'+'part_4_'+pass_nr+'.nc')

In [ ]:
for pass_nr in passes: 
    print(pass_nr)
    dataset_part_5 = part_5_NetCDF(path_merged,pass_nr)
    dataset_part_5.to_netcdf(save_path+'part_5/'+'part_5_'+pass_nr+'.nc')
    print(save_path+'part_5/'+'part_5_'+pass_nr+'.nc')

In [ ]:
save_path = '/uio/hypatia/geofag-personlig/students/metos/ingvilob/Investigation_part_1/'
pass_nr = 'pass_003'
ds = xr.open_dataset(save_path+'part_1/'+'part_1_'+pass_nr+'.nc')

In [ ]:
save_path = '/uio/hypatia/geofag-personlig/students/metos/ingvilob/Investigation_part_1/'
pass_nr = 'pass_003'
ds = xr.open_dataset(save_path+'part_2/'+'part_2_'+pass_nr+'.nc')

In [ ]:
save_path = '/uio/hypatia/geofag-personlig/students/metos/ingvilob/Investigation_part_1/'
pass_nr = 'pass_003'
ds = xr.open_dataset(save_path+'part_3/'+'part_3_'+pass_nr+'.nc')

In [ ]:
save_path = '/uio/hypatia/geofag-personlig/students/metos/ingvilob/Investigation_part_1/'
pass_nr = 'pass_003'
ds = xr.open_dataset(save_path+'part_4/'+'part_4_'+pass_nr+'.nc')

In [ ]:
save_path = '/uio/hypatia/geofag-personlig/students/metos/ingvilob/Investigation_part_1/'
pass_nr = 'pass_003'
ds = xr.open_dataset(save_path+'part_5/'+'part_5_'+pass_nr+'.nc')

In [ ]:
ds

## Unravel and create CSV-files: 

In [ ]:
passes = ['pass_003', 'pass_005', 'pass_007', 'pass_008', 'pass_009', 'pass_010', 'pass_012', 'pass_014', 'pass_016', 'pass_018',
          'pass_031', 'pass_033', 'pass_035', 'pass_036', 'pass_037', 'pass_038', 'pass_040', 'pass_042', 'pass_044', 'pass_046',
          'pass_059', 'pass_061', 'pass_063', 'pass_064', 'pass_065', 'pass_066', 'pass_068', 'pass_070', 'pass_072', 'pass_074',
          'pass_087', 'pass_089', 'pass_091', 'pass_092', 'pass_093', 'pass_094', 'pass_096', 'pass_098', 'pass_100',
          'pass_113', 'pass_115', 'pass_117', 'pass_119', 'pass_120', 'pass_121', 'pass_122', 'pass_124', 'pass_126', 'pass_128',
          'pass_141', 'pass_143', 'pass_145', 'pass_147', 'pass_148', 'pass_149', 'pass_150', 'pass_152', 'pass_154', 'pass_156',
          'pass_169', 'pass_171', 'pass_173', 'pass_175', 'pass_176', 'pass_177', 'pass_178', 'pass_180', 'pass_182', 'pass_184',
          'pass_197', 'pass_199', 'pass_201', 'pass_203', 'pass_204', 'pass_205', 'pass_206', 'pass_208', 'pass_210', 'pass_212',
          'pass_225', 'pass_227', 'pass_229', 'pass_231', 'pass_232', 'pass_233', 'pass_234', 'pass_236', 'pass_238', 'pass_240',
          'pass_253', 'pass_255', 'pass_257', 'pass_259', 'pass_260', 'pass_262', 'pass_264', 'pass_266', 'pass_268',
          'pass_281', 'pass_283', 'pass_285', 'pass_286', 'pass_287', 'pass_288', 'pass_290', 'pass_292', 'pass_294', 'pass_296',
          'pass_309', 'pass_311', 'pass_313', 'pass_314', 'pass_315', 'pass_316', 'pass_318', 'pass_320', 'pass_322', 'pass_324',
          'pass_337', 'pass_339', 'pass_341', 'pass_342', 'pass_343', 'pass_344', 'pass_346', 'pass_348', 'pass_350', 'pass_352',
          'pass_365', 'pass_367', 'pass_369', 'pass_370', 'pass_371', 'pass_372', 'pass_374', 'pass_376', 'pass_378', 'pass_391',
          'pass_393', 'pass_395', 'pass_397', 'pass_398', 'pass_399', 'pass_400', 'pass_402', 'pass_404', 'pass_406', 'pass_419',
          'pass_421', 'pass_423', 'pass_425', 'pass_426', 'pass_427', 'pass_428', 'pass_430', 'pass_432', 'pass_434', 'pass_447',
          'pass_449', 'pass_451', 'pass_453', 'pass_454', 'pass_455', 'pass_456', 'pass_458', 'pass_460', 'pass_462', 'pass_475',
          'pass_477', 'pass_479', 'pass_481', 'pass_482', 'pass_483', 'pass_484', 'pass_486', 'pass_488', 'pass_490', 'pass_503',
          'pass_505', 'pass_507', 'pass_509', 'pass_510', 'pass_511', 'pass_512', 'pass_514', 'pass_516', 'pass_518', 'pass_531',
          'pass_533', 'pass_535', 'pass_536', 'pass_537', 'pass_538', 'pass_539', 'pass_540', 'pass_542', 'pass_544', 'pass_546', 'pass_559',
          'pass_561', 'pass_563', 'pass_564', 'pass_565', 'pass_566', 'pass_568', 'pass_570', 'pass_572', 'pass_574']

In [ ]:
save_path = '/uio/hypatia/geofag-personlig/students/metos/ingvilob/Investigation_part_1/'

In [ ]:
def unravel_values_part_1(files_ds):    
    # GET VALUES :
    c1     = files_ds['count_ssh_karin']
    c2     = files_ds['count_ssh_karin_2']
    std    = files_ds['std_ssh_karin_2']
    meant  = files_ds['mean_ssh_karin_2_tides']
    means  = files_ds['mean_slope_ssh_karin_2_tides']
    meanb  = files_ds['mean_slope_bathy']
    meanb2 = files_ds['mean_slope_bathy_norm']
    mdtm   = files_ds['mean_dynamic_topography_model']
    lat    = files_ds['latitude']
    lon    = files_ds['longitude']
    # FIX LONGITUDE (IF NEEDED) : 
    #lon = np.where(lon>180.0,lon-360,lon)
    # UNRAVEL : 
    c1     = np.ravel(c1)
    c2     = np.ravel(c2)
    std    = np.ravel(std)
    meant  = np.ravel(meant)
    means  = np.ravel(means)
    meanb  = np.ravel(meanb)
    meanb2 = np.ravel(meanb2)
    mdtm   = np.ravel(mdtm)
    lat    = np.ravel(lat)
    lon    = np.ravel(lon)
    # Return values :
    return c1, c2, std, meant, means, meanb, meanb2, mdtm, lat, lon

def unravel_values_part_2(files_ds):    
    # GET VALUES :
    std   = files_ds['std_ssha_karin_2']
    mean  = files_ds['mean_ssha_karin_2']
    eke1  = files_ds['eke_ssha_karin_2_normal']
    eke2  = files_ds['eke_ssha_karin_2_option_2']
    lat   = files_ds['latitude']
    lon   = files_ds['longitude']
    # FIX LONGITUDE (IF NEEDED) : 
    #lon = np.where(lon>180.0,lon-360,lon)
    # UNRAVEL : 
    std   = np.ravel(std)
    mean  = np.ravel(mean)
    eke1  = np.ravel(eke1)
    eke2  = np.ravel(eke2)
    lat   = np.ravel(lat)
    lon   = np.ravel(lon)
    # Return values :
    return std, mean, eke1, eke2, lat, lon

def unravel_values_part_3(files_ds):    
    # GET VALUES :
    mean  = files_ds['mean_ssha_karin_2_W_S']
    speed = files_ds['speed_ssha_karin_2_W_S']
    ug    = files_ds['ug_w_s_r']
    vg    = files_ds['vg_w_s_r']
    eke   = files_ds['eke_ssha_karin_2_W_S']
    lat   = files_ds['latitude']
    lon   = files_ds['longitude']
    # FIX LONGITUDE (IF NEEDED) : 
    #lon = np.where(lon>180.0,lon-360,lon)
    # UNRAVEL : 
    mean  = np.ravel(mean)
    speed = np.ravel(speed)
    ug    = np.ravel(ug)
    vg    = np.ravel(vg)
    eke   = np.ravel(eke)
    lat   = np.ravel(lat)
    lon   = np.ravel(lon)
    # Return values :
    return mean, speed, ug, vg, eke, lat, lon

def unravel_values_part_4(files_ds):   
    # GET VALUES :
    stdugg = files_ds['std_ug_zomer']
    stdvgg = files_ds['std_vg_zomer']
    stdugt = files_ds['std_ug_track']
    stdvgt = files_ds['std_vg_track']
    lat    = files_ds['latitude']
    lon    = files_ds['longitude']
    # FIX LONGITUDE (IF NEEDED) : 
    #lon = np.where(lon>180.0,lon-360,lon)
    # UNRAVEL : 
    stdugg = np.ravel(stdugg)
    stdvgg = np.ravel(stdvgg)
    stdugt = np.ravel(stdugt)
    stdvgt = np.ravel(stdvgt)
    lat    = np.ravel(lat)
    lon    = np.ravel(lon)
    # Return values :
    return stdugg, stdvgg, stdugt, stdvgt, lat, lon

def unravel_values_part_5(files_ds):   
    # GET VALUES :
    ms   = files_ds['mean_speed']
    stds = files_ds['std_speed']
    cvs  = files_ds['CV_speed']
    lat  = files_ds['latitude']
    lon  = files_ds['longitude']
    # FIX LONGITUDE (IF NEEDED) : 
    #lon = np.where(lon>180.0,lon-360,lon)
    # UNRAVEL : 
    ms   = np.ravel(ms)
    stds = np.ravel(stds)
    cvs  = np.ravel(cvs)
    lat  = np.ravel(lat)
    lon  = np.ravel(lon)
    # Return values :
    return ms, stds, cvs, lat, lon

In [ ]:
def create_dataset_1(save_path,pass_nr):
    # Get NetCDF file : 
    files_ds = xr.open_dataset(save_path+'part_1/'+'part_1_'+pass_nr+'.nc')
    # Unravel data : 
    c1, c2, std, meant, means, meanb, meanb2, mdtm, lat, lon = unravel_values_part_1(files_ds)
    # CREATE DATAFRAME :
    df = pd.DataFrame({'count_ssh_karin'                : c1, 
                       'count_ssh_karin_2'              : c2, 
                       'std_ssh_karin_2'                : std, 
                       'mean_ssh_karin_2_tides'         : meant, 
                       'mean_slope_ssh_karin_2_tides'   : means, 
                       'mean_slope_bathy'               : meanb, 
                       'mean_slope_bathy_norm'          : meanb2, 
                       'mean_dynamic_topography_model'  : mdtm, 
                       'lat'                            : lat, 
                       'lon'                            : lon})
    # Return dataset :
    return df

def create_dataset_2(save_path,pass_nr):
    # Get NetCDF file : 
    #files_ds = xr.open_dataset(save_path+'part_2/'+'part_2_'+pass_nr+'.nc')
    files_ds = xr.open_dataset(save_path+'part_2_sm_2/'+'part_2_'+pass_nr+'.nc')
    # Unravel data : 
    std, mean, eke1, eke2, lat, lon = unravel_values_part_2(files_ds)
    # CREATE DATAFRAME :
    df = pd.DataFrame({'std_ssha_karin_2'          : std, 
                       'mean_ssha_karin_2'         : mean, 
                       'eke_ssha_karin_2_normal'   : eke1, 
                       'eke_ssha_karin_2_option_2' : eke2, 
                       'lat'                       : lat, 
                       'lon'                       : lon})
    # Return dataset :
    return df

def create_dataset_3(save_path,pass_nr):
    # Get NetCDF file : 
    #files_ds = xr.open_dataset(save_path+'part_3/'+'part_3_'+pass_nr+'.nc')
    files_ds = xr.open_dataset(save_path+'part_3_sm_2/'+'part_3_'+pass_nr+'.nc')
    # Unravel data : 
    mean, speed, ug, vg, eke, lat, lon = unravel_values_part_3(files_ds)
    # CREATE DATAFRAME :
    df = pd.DataFrame({'mean_ssha_karin_2_W_S'  : mean, 
                       'speed_ssha_karin_2_W_S' : speed, 
                       'ug_w_s_r'               : ug, 
                       'vg_w_s_r'               : vg, 
                       'eke_ssha_karin_2_W_S'   : eke, 
                       'lat'                    : lat, 
                       'lon'                    : lon})
    # Return dataset :
    return df

def create_dataset_4(save_path,pass_nr):
    # Get NetCDF file : 
    files_ds = xr.open_dataset(save_path+'part_4/'+'part_4_'+pass_nr+'.nc')
    # Unravel data : 
    stdugg, stdvgg, stdugt, stdvgt, lat, lon = unravel_values_part_4(files_ds)
    # CREATE DATAFRAME :
    df = pd.DataFrame({'std_ug_zomer' : stdugg, 
                       'std_vg_zomer' : stdvgg, 
                       'std_ug_track' : stdugt, 
                       'std_vg_track' : stdvgt, 
                       'lat'          : lat, 
                       'lon'          : lon})
    # Return dataset :
    return df

def create_dataset_5(save_path,pass_nr):
    # Get NetCDF file : 
    files_ds = xr.open_dataset(save_path+'part_5/'+'part_5_'+pass_nr+'.nc')
    # Unravel data : 
    ms, stds, cvs, lat, lon = unravel_values_part_5(files_ds)
    # CREATE DATAFRAME :
    df = pd.DataFrame({'mean_speed' : ms, 
                       'std_speed'  : stds, 
                       'CV_speed'   : cvs, 
                       'lat'        : lat, 
                       'lon'        : lon})
    # Return dataset :
    return df

In [ ]:
for pass_nr in passes:
    print('Pass number : ', pass_nr)
    # Get datasets : 
    df = create_dataset_1(save_path,pass_nr)
    # Save the dataframe to a csv file
    csv_name  = 'part_1_'+pass_nr+'.csv'
    # FILE PATHS :
    save_to_csv = '/uio/hypatia/geofag-personlig/students/metos/ingvilob/Investigation_part_1/part_1_csv/'+csv_name
    # SAVE :
    df.to_csv(save_to_csv, index=False)

In [ ]:
for pass_nr in passes:
    print('Pass number : ', pass_nr)
    # Get datasets : 
    df = create_dataset_2(save_path,pass_nr)
    # Save the dataframe to a csv file
    csv_name  = 'part_2_'+pass_nr+'.csv'
    # FILE PATHS :
    #save_to_csv = '/uio/hypatia/geofag-personlig/students/metos/ingvilob/Investigation_part_1/part_2_csv/'+csv_name
    save_to_csv = '/uio/hypatia/geofag-personlig/students/metos/ingvilob/Investigation_part_1/part_2_csv_sm_2/'+csv_name
    # SAVE :
    df.to_csv(save_to_csv, index=False)

In [ ]:
for pass_nr in passes:
    print('Pass number : ', pass_nr)
    # Get datasets : 
    df = create_dataset_3(save_path,pass_nr)
    # Save the dataframe to a csv file
    csv_name  = 'part_3_'+pass_nr+'.csv'
    # FILE PATHS :
    #save_to_csv = '/uio/hypatia/geofag-personlig/students/metos/ingvilob/Investigation_part_1/part_3_csv/'+csv_name
    save_to_csv = '/uio/hypatia/geofag-personlig/students/metos/ingvilob/Investigation_part_1/part_3_csv_sm_2/'+csv_name
    #save_to_csv = '/uio/hypatia/geofag-personlig/students/metos/ingvilob/Investigation_part_1/Smoothed_1_time/part_3_csv/'+csv_name
    # SAVE :
    df.to_csv(save_to_csv, index=False)

In [ ]:
for pass_nr in passes:
    print('Pass number : ', pass_nr)
    # Get datasets : 
    df = create_dataset_4(save_path,pass_nr)
    # Save the dataframe to a csv file
    csv_name  = 'part_4_'+pass_nr+'.csv'
    # FILE PATHS :
    save_to_csv = '/uio/hypatia/geofag-personlig/students/metos/ingvilob/Investigation_part_1/part_4_csv/'+csv_name
    # SAVE :
    df.to_csv(save_to_csv, index=False)

In [ ]:
for pass_nr in passes:
    print('Pass number : ', pass_nr)
    # Get datasets : 
    df = create_dataset_5(save_path,pass_nr)
    # Save the dataframe to a csv file
    csv_name  = 'part_5_'+pass_nr+'.csv'
    # FILE PATHS :
    save_to_csv = '/uio/hypatia/geofag-personlig/students/metos/ingvilob/Investigation_part_1/part_5_csv/'+csv_name
    # SAVE :
    df.to_csv(save_to_csv, index=False)